## 6. 공간으로 시간벌기

- 정렬, 문자열 매칭, 해싱

### 6.1 기수 정렬

- 지금까지와는 달리, 비교 없이 정렬(기수, 카운팅 정렬)
- 버킷과 Queue를 사용, 각 자릿수별로 순차적으로 정렬
- But 키값이 자연수여야 하고, 한글이나 한자에는 적용 어려움

In [1]:
from queue import Queue
def radix_sort(A):
    queues = []
    for i in range(BUCKETS:=10):
        queues.append(Queue())

    n = len(A)
    factor = 1
    for d in range(DIGITS:=4):
        for i in range(n):
            queues[(A[i]//factor)%10].put(A[i])
        j = 0
        for b in range(BUCKETS):
            while not queues[b].empty():
                A[j] = queues[b].get()
                j += 1
        factor *= 10
        print("step", d+1, A)


import random
random.seed(23)
BUCKETS = 10
DIGITS = 4
data = []
for i in range(10):
    data.append(random.randint(1,9999))
radix_sort(data)


step 1 [4750, 280, 9700, 6943, 6213, 8685, 2135, 5026, 5867, 1369]
step 2 [9700, 6213, 5026, 2135, 6943, 4750, 5867, 1369, 280, 8685]
step 3 [5026, 2135, 6213, 280, 1369, 8685, 9700, 4750, 5867, 6943]
step 4 [280, 1369, 2135, 4750, 5026, 5867, 6213, 6943, 8685, 9700]


### 6.2 카운팅 정렬

- 각 항목의 빈도수를 센 다음에, 그 횟수만큼 출력한다
- 수의 정렬에는 매우 힘들고, 킷값이 일정한 개수로 제한되는 문자열같은 경우 효과적


In [6]:
def counting_sort(A):
    output = [0] * len(A)
    count = [0] * MAX_VAL

    for i in A:
        count[i] += 1
    
    for i in range(MAX_VAL):
        count[i] += count[i-1]

    for i in range(len(A)):
        output[count[A[i]-1]] = A[i]
        count[A[i]] -= 1
    
    for i in range(len(A)):
        A[i] = output[i]
    
    return A


MAX_VAL = 10
data = [1, 4, 1, 2, 7, 5, 2]
counting_sort(data)

[2, 0, 0, 0, 5, 0, 7]

### 6.3 문자열 매칭

- 전처리 과정을 통해 패턴에 대한 정보를 얻어 테이블에 저장후, 패턴 매칭시 사용하자는 아이디어 등장

#### 6.3.1 Horspool 알고리즘

- 휴리스틱 알고리즘
- 문자를 뒤에서부터 검사, 맞지많으면 패틴길이만큼 뛰어넘을 수 있음!
- 불일치문자임에도 패턴에 포함된 글자라면, 패턴과 (현재 불일치하는) 텍스트의 위치를 맞춰줌!
<hr>

- 이를 위해, shift table 생성 // 
    패턴이 banana일 경우,
    - tbl[m] = tbl[o] = 6
    - tbl[b] = 6-1-pos[b]
    

In [13]:
#shift table
NO_OF_CHARS = 128
def shift_tbl(pat):
    m = len(pat)
    tbl = [m]*NO_OF_CHARS

    for i in range(m-1):
        tbl[ord(pat[i])] = m-1-i

    return tbl


In [17]:
def search_horspool(T, P):
    m = len(P)
    n = len(T)
    t = shift_tbl(P)
    i = m-1
    while i <= n-1:
        k=0
        while k<=m-1 and P[m-1-k]==T[i-k]:
            k+=1
        if k == m:
            return i-m+1
        else:
            tc = t[ord(T[i-k])]
            i += tc-k
    return -1

search_horspool("APPLEMANGOBANANAGRAPE","BANANA")

10

#### 6.3.2 Boyer-Moore 알고리즘

### 6.4 해싱(Hashing)
- 최적의 경우 O(1)까지! 공간을 이용해 시간 효율성을 높이는 대표적인 예시
- 해시 함수
    - 탐색키를 입력받아 해시주소를 계산
    - 삽입, 삭제, 탐색 연산은 모두 이 주소에서 이뤄짐
- 해싱 과정에서 충돌이 발생하면 여러 슬롯에 저장하면 되지만, 슬롯보다 많이 발생하면 overflow 발생



#### 6.4.1 선형 조사법(linear probing)에 의한 오버플로 처리
- 버킷에 빈 슬롯이 없을 경우 순차적으로 빈 슬롯 조사
- 선형 조사법은 충돌이 연속적으로 집중됨... clustering
- 삭제의 경우, 빈 슬롯을 "완전 새거/있다가 지워진거" 두가지로 구별해야 한다. (충돌로 뒤로 밀려난 레코드를 찾을수가 없어지므로)

In [18]:
M = 13
table = [None]*M

def hashFn(key):
    return key%M

#선형 조사법의 삽입 알고리즘
def lp_insert(key):
    id = hashFn(key)
    count = M
    while count>0 and (table[id]!= None and table[id]!= -1):
        id = (id+1+M)%M
        count -= 1
    if count > 0:
        table[id] = key
    return

#선형 조사법의 탐색 알고리즘
def lp_search(key):
    id = hashFn(key)
    count = M
    while count>0:
        if table[id] == None:
            return None
        if table[id] != -1 and table[id] == key:
            return table[id]
        id = (id+1+M)%M
        count -= 1
    return None

#삭제 알고리즘
def lp_delete(key):
    id = hashFn(key)
    count = M
    while count>0:
        if table[id] == None: return
        if table[id] != -1 and table[id] == key:        #None과 -1로 구별
            table[id] = -1
            return
    id = (id+1+M)%M
    count -= 1

In [19]:
print("start:", table)
lp_insert(45); print("45 삽입", table)
lp_insert(27); print("27 삽입", table)
lp_insert(88); print("88 삽입", table)
lp_insert(9); print("9 삽입", table)
lp_insert(71); print("71 삽입", table)
lp_insert(60); print("60 삽입", table)
lp_insert(46); print("46 삽입", table)
lp_insert(38); print("38 삽입", table)
lp_insert(24); print("24 삽입", table)
lp_delete(60); print("60 삭제", table)
print("46 탐색", lp_search(46))

start: [None, None, None, None, None, None, None, None, None, None, None, None, None]
45 삽입 [None, None, None, None, None, None, 45, None, None, None, None, None, None]
27 삽입 [None, 27, None, None, None, None, 45, None, None, None, None, None, None]
88 삽입 [None, 27, None, None, None, None, 45, None, None, None, 88, None, None]
9 삽입 [None, 27, None, None, None, None, 45, None, None, 9, 88, None, None]
71 삽입 [None, 27, None, None, None, None, 45, 71, None, 9, 88, None, None]
60 삽입 [None, 27, None, None, None, None, 45, 71, 60, 9, 88, None, None]
46 삽입 [None, 27, None, None, None, None, 45, 71, 60, 9, 88, 46, None]
38 삽입 [None, 27, None, None, None, None, 45, 71, 60, 9, 88, 46, 38]
24 삽입 [24, 27, None, None, None, None, 45, 71, 60, 9, 88, 46, 38]
60 삭제 [24, 27, None, None, None, None, 45, 71, -1, 9, 88, 46, 38]
46 탐색 46


+)이차 조사법(quadratic probing)

- clustering의 완화... 2차 집중이 발생하지만 1차 집중보다 낫다


<br><b><center>id = (id+i*i)%M</center>

#### 6.4.2 Chaining에 의한 오버플로 정리
- linked list를 통해 무한정 추가 가능. 충돌 시 새로운 노드 생성 및 저장

#### 6.4.3 해시 함수
- 충돌이 적고, 주소가 고르게 분포되며, 계산이 빠를수록 좋은 함수
    - 제산 함수(k mod M, 나머지 계산)
        - 이때, M은 prime number를 선택(고른 분포를 위해)
    - 폴딩 함수
        - 탐색키가 테이블의 크기보다 더 큰 정수일 경우, 탐색키를 몇 개의 부분으로 나누어, 더하거나, 비트별 XOR 연산을 이용하는 것
            - shift folding -> 여러 부분으로 나눈 것을 더함
            - boundary folding -> 이웃한 부분을 거꾸로 해서 더함
    - 중간 제곱 함수
        - 탐색키를 제곱해서 중간의 몇 비트를 취해서 해시 주소를 생성함.
        - 제곱한 값의 중간 비트들은 대개 탐색키의 모든 자리의 숫자들과 관련이 있으며, 보통 비교적 고르게 분산됨.
    - 비트 추출 방법
        - 해시테이블 사이즈가 2의 제곱수일 때, 탐색키를 이진수로 간주하여 임의의 위치의 k개의 비트를 해시 주소로 사용(탐색키의 일부 정보만 사용하기에 주소 집중될 가능성 높다)
    - 숫자 분석 방법
        - 숫자의 각 위치별 특성이 있을 경우 유용함. (ex. 학번의 2022123456에서 2022는 중복이 높으므로 사용하지 않는다거나)

    - 탐색키가 문자열일 경우
        - 각 문자에 정수를 대응시켜 바꾸거나, 문자열 안의 모든 문자를 골고루 사용함

In [20]:
#문자열일 경우 해시함수의 예시
def hashFn_str(key):
    sum = 0
    for c in key:
        sum += ord(c)
    return sum%M


Practice